In [15]:
import pandas as pd
from datetime import datetime, timedelta
import sys

def analyze_excel_file(file_path, output_file):
    # Load the Excel file into a pandas DataFrame
    try:
        df = pd.read_excel(file_path)
    except Exception as e:
        print(f"Error reading the Excel file: {e}")
        return

    # Convert 'Time' and 'Time Out' columns to datetime objects
    df['Time'] = pd.to_datetime(df['Time'], format='%m/%d/%Y %I:%M %p', errors='coerce')
    df['Time Out'] = pd.to_datetime(df['Time Out'], format='%m/%d/%Y %I:%M %p', errors='coerce')

    # Convert 'Timecard Hours (as Time)' column to timedelta objects
    df['Timecard Hours (as Time)'] = pd.to_timedelta(
        df['Timecard Hours (as Time)'].fillna('0:00').astype(str).str.split(':').apply(
            lambda x: timedelta(hours=int(x[0]) if len(x) > 0 else 0, minutes=int(x[1]) if len(x) > 1 else 0, seconds=int(x[2]) if len(x) > 2 else 0)
        )
    )

    # Sort the DataFrame by 'Employee Name' and 'Time'
    df.sort_values(by=['Employee Name', 'Time'], inplace=True)

    # Function to calculate consecutive days worked
    def consecutive_days(series):
        return (series - series.shift(1)).dt.days == 1 if len(series) > 1 else False

    # Function to check time between shifts
    def time_between_shifts(series):
        return ((series - series.shift(1)) < timedelta(hours=10)) & ((series - series.shift(1)) > timedelta(hours=1)) if len(series) > 1 else False

    # Function to identify long shifts
    def long_shift(series):
        return series > timedelta(hours=14)

    # Redirect print output to a file
    original_stdout = sys.stdout
    with open(output_file, 'a') as f:  # Use 'a' to append to the file
        sys.stdout = f

        # Print the results
        print("\n-----------------------------------")
        print("Employees who worked for 7 consecutive days:")
        print(df[consecutive_days(df.groupby('Employee Name')['Time'].transform(lambda x: x))][['Employee Name', 'Position ID']].drop_duplicates().to_string(index=False))

        print("\n-----------------------------------")
        print("Employees with less than 10 hours between shifts but greater than 1 hour:")
        print(df[time_between_shifts(df.groupby('Employee Name')['Time'].transform(lambda x: x))][['Employee Name', 'Position ID']].drop_duplicates().to_string(index=False))

        print("\n-----------------------------------")
        print("Employees who worked for more than 14 hours in a single shift:")
        print(df[long_shift(df['Timecard Hours (as Time)'])][['Employee Name', 'Position ID']].to_string(index=False))

    # Reset the standard output
    sys.stdout = original_stdout


file_path = "Assignment_Timecard.xlsx"
output_file = "output.txt"
analyze_excel_file(file_path, output_file)
